### Load Package

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Activation, Flatten, Concatenate, Embedding, Dot, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import RootMeanSquaredError

### Load dataset

- use movie lense public dataset

In [2]:
data = Dataset.load_builtin('ml-100k')

In [3]:
ratings = pd.DataFrame(data.raw_ratings, columns = ['user', 'item', 'rate',' id'])

print(ratings.shape)
ratings.head()

(100000, 4)


,user,item,rate,id
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


### Score Mixin

In [4]:
class ScoreMixin(object):
    
    def score(self):
        print(f"{self.__class__.__name__} evaluation start.")
        rating_test = self.ratings.sample(n=round(self.ratings.shape[0]*0.25))
        y_true = rating_test.rate
        y_pred = [self.rate(user, item) for user, item in zip(rating_test.user, rating_test.item)]
        rmse_score = self.rmse(y_true, y_pred)
        print(f"{self.__class__.__name__}", rmse_score)
        
        return rmse_score
    
    @staticmethod
    def rmse(y_true, y_pred):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        
        return rmse

### Baseline Recommend

In [5]:
class Baseline_Recommender(ScoreMixin):
    
    def train(self, X):
        print(f"{self.__class__.__name__} train start.")
        self.ratings = X.copy()
        self.full_matrix = self.ratings.pivot('user', 'item', 'rate')
        self.full_matrix_dummy = self.full_matrix.fillna(0).copy()
        self.avg_rating = self.ratings.rate.mean()
        
        return self
    
    def rate(self, user, item):
        return self.avg_rating
    
    def recommend(self, user, n_items):
        print(f"{self.__class__.__name__} recommend start.")
        try:
            user_items = self.full_matrix.loc[user].copy()
        except: # recommend to random item for new customer
            reco_items = self.ratings.item.sample(n=n_items, random_state=42).tolist()
        else:
            for item in self.full_matrix.columns:
                if pd.notnull(user_items[item]):
                    user_items.loc[item] = 0 # already rating
                else:
                    user_items.loc[item] = self.rate(user, item)
                    
            reco_items = user_items.sort_values(ascending=False).index[:n_items].tolist()
            
        return reco_items

In [6]:
br = Baseline_Recommender()

In [7]:
br.train(ratings)

Baseline_Recommender train start.


In [8]:
br.rate('1', '947')

3.52986

In [9]:
br.recommend('1', 10)

Baseline_Recommender recommend start.


['999', '372', '350', '818', '349', '348', '347', '346', '345', '344']

### UBCF (User Based Collaborative Filtering)

In [10]:
class Ubcf_Recommender(Baseline_Recommender):
    
    def train(self, X):
        super().train(X)
        self.user_sim = pd.DataFrame(data=cosine_similarity(self.full_matrix_dummy, self.full_matrix_dummy), 
                                     index=self.full_matrix_dummy.index,
                                     columns=self.full_matrix_dummy.index)
        
        return self
    
    def rate(self, user, item):
        if item in self.full_matrix.columns:
            user_sims = self.user_sim[user]
            item_ratings = self.full_matrix[item]
            none_ratings_idx = item_ratings[item_ratings.isnull()].index
            user_sims = user_sims.drop(none_ratings_idx)
            item_ratings = item_ratings.dropna()    
            rating = np.dot(user_sims, item_ratings) / user_sims.sum()
        else:
            rating = self.avg_rating
            
        return rating
        

In [11]:
ur = Ubcf_Recommender()

In [12]:
ur.train(ratings)

Ubcf_Recommender train start.


In [13]:
ur.rate('1', '947')

3.316926723993723

In [14]:
ur.recommend('1', 10)

Ubcf_Recommender recommend start.


['1201', '814', '1653', '1467', '1122', '1536', '1500', '1599', '1189', '1293']

### UBCF KNN (K-Nearest Neighbor)

In [15]:
class Ubcf_Knn_Recommender(Ubcf_Recommender):
    
    def __init__(self, K=30):
        self.K = K
                
    def rate(self, user, item):
        if item in self.full_matrix.columns:
            user_sims = self.user_sim[user]
            item_ratings = self.full_matrix[item]
            none_ratings_idx = item_ratings[item_ratings.isnull()].index
            user_sims = user_sims.drop(none_ratings_idx)
            item_ratings = item_ratings.dropna()    

            if self.K == 0:
                rating = np.dot(user_sims, item_ratings) / user_sims.sum()
            else:
                if len(user_sims) > 1:
                    neighbor_size = min(len(user_sims), self.K)
                    user_sims = np.array(user_sims)
                    item_ratings = np.array(item_ratings)
                    user_idx = np.argsort(user_sims)
                    user_sims = user_sims[user_idx][-neighbor_size:]
                    item_ratings = item_ratings[user_idx][-neighbor_size:]
                    rating = np.dot(user_sims, item_ratings) / user_sims.sum()
                else:
                    rating = self.avg_rating
                    
        else:
            rating = self.avg_rating
            
        return rating

In [16]:
ukr = Ubcf_Knn_Recommender(10)

In [17]:
ukr.train(ratings)

Ubcf_Knn_Recommender train start.


In [18]:
ukr.rate('1', '947')

3.4598709191134143

In [19]:
ukr.recommend('1', 10)

Ubcf_Knn_Recommender recommend start.


['1500', '1467', '1293', '1189', '1656', '483', '313', '963', '1449', '357']

### UBCF KNN Bias

In [20]:
class Ubcf_Knn_Bias_Recommender(Ubcf_Knn_Recommender):
        
    def train(self, X):
        super().train(X)
        self.ratings_mean = self.full_matrix.mean(axis=1)
        self.ratings_bias = (self.full_matrix.T - self.ratings_mean).T
        
        return self
    
    def rate(self, user, item):
        if item in self.ratings_bias.columns:
            user_sims = self.user_sim[user]
            item_ratings = self.ratings_bias[item]
            none_ratings_idx = item_ratings[item_ratings.isnull()].index
            user_sims = user_sims.drop(none_ratings_idx)
            item_ratings = item_ratings.dropna()    

            if self.K == 0:
                rating = np.dot(user_sims, item_ratings) / user_sims.sum() + self.ratings_mean[user]
            else:
                if len(user_sims) > 1:
                    neighbor_size = min(len(user_sims), self.K)
                    user_sims = np.array(user_sims)
                    item_ratings = np.array(item_ratings)
                    user_idx = np.argsort(user_sims)
                    user_sims = user_sims[user_idx][-neighbor_size:]
                    item_ratings = item_ratings[user_idx][-neighbor_size:]
                    rating = np.dot(user_sims, item_ratings) / user_sims.sum() + self.ratings_mean[user]
                else:
                    rating = self.ratings_mean[user]
                    
        else:
            rating = self.ratings_mean[user]
            
        return rating

In [21]:
ukbr = Ubcf_Knn_Bias_Recommender()

In [22]:
ukbr.train(ratings)

Ubcf_Knn_Bias_Recommender train start.


In [23]:
ukbr.rate('1', '947')

3.686307852032466

In [24]:
ukbr.recommend('1', 10)

Ubcf_Knn_Bias_Recommender recommend start.


['1500', '1467', '1463', '1293', '1642', '1367', '1449', '1398', '1189', '408']

### UBCF KNN Bias Confidence level

In [25]:
class Ubcf_Knn_Bias_Conf_Recommender(Ubcf_Knn_Bias_Recommender):
    
    def __init__(self, K=30, rating_cnt=2, min_rating=2):
        super().__init__(K)
        self.RATING_CNT = rating_cnt
        self.MIN_RATING = min_rating
        
    def train(self, X):
        super().train(X)
        rating_binary1 = np.array(self.full_matrix > 0).astype(float)
        rating_binary2 = rating_binary1.T
        counts = np.dot(rating_binary1, rating_binary2)
        self.counts = pd.DataFrame(counts, index=self.full_matrix.index, columns=self.full_matrix.index).fillna(0)
        
    def rate(self, user, item):
        
        if item in self.ratings_bias.columns:
            user_sims = self.user_sim[user]
            item_ratings = self.ratings_bias[item]
            
            no_ratings = item_ratings.isnull()
            common_counts = self.counts[user]
            low_significance = common_counts < self.RATING_CNT
            none_rating_idx = item_ratings[no_ratings | low_significance].index
            
            user_sims = user_sims.drop(none_rating_idx)
            item_ratings = item_ratings.dropna()
            
            if self.K == 0:
                rating = np.dot(user_sims, item_ratings) / user_sims.sum() + self.ratings_mean[user]
            else:
                if len(user_sims) > self.MIN_RATING:
                    neighbor_size = min(len(user_sims), self.K)
                    user_sims = np.array(user_sims)
                    item_ratings = np.array(item_ratings)
                    user_idx = np.argsort(user_sims)
                    user_sims = user_sims[user_idx][-neighbor_size:]
                    item_ratings =item_ratings[user_idx][-neighbor_size:]
                    rating = np.dot(user_sims, item_ratings) / user_sims.sum() + self.ratings_mean[user]
                else:
                    rating = self.ratings_mean[user]
        else:
            rating = self.ratings_mean[user]
        
        return rating
                    

In [26]:
ukbcr = Ubcf_Knn_Bias_Conf_Recommender()

In [27]:
ukbcr.train(ratings)

Ubcf_Knn_Bias_Conf_Recommender train start.


In [28]:
ukbcr.rate('1', '947')

3.686307852032466

In [29]:
ukbcr.recommend('1', 10)

Ubcf_Knn_Bias_Conf_Recommender recommend start.


['1463', '1293', '1367', '1449', '1189', '408', '318', '1643', '483', '357']

### IBCF (Item Based Collaborative Filtering)

In [30]:
class Ibcf_Recommender(Baseline_Recommender):
    
    def train(self, X):
        super().train(X)
        self.full_matrix_dummy = self.full_matrix_dummy.T
        self.item_sim = pd.DataFrame(data=cosine_similarity(self.full_matrix_dummy, self.full_matrix_dummy),
                                     index=self.full_matrix_dummy.index,
                                     columns=self.full_matrix_dummy.index)
        
        return self
    
    def rate(self, user, item):
        if item in self.full_matrix.columns:
            item_sims = self.item_sim[item]
            item_ratings = self.full_matrix.T[user]
            none_ratings_idx = item_ratings[item_ratings.isnull()].index
            item_sims = item_sims.drop(none_ratings_idx)
            item_ratings = item_ratings.dropna()
            rating = np.dot(item_sims, item_ratings) / item_sims.sum()
        else:
            rating = self.avg_rating
        
        return rating

In [31]:
ir = Ibcf_Recommender()

In [32]:
ir.train(ratings)

Ibcf_Recommender train start.


In [33]:
ir.rate('1', '947')

3.83094861122187

In [34]:
ir.recommend('1', 10)

Ibcf_Recommender recommend start.


['1156',
 '1601',
 '1656',
 '1593',
 '1674',
 '1347',
 '1618',
 '1104',
 '1332',
 '1507']

### MF (Matrix Factorization)

In [35]:
class MF_Rrecommender(Baseline_Recommender):
    
    def __init__(self, K, alpha, beta, iter, verbose):
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iter = iter
        self.verbose = verbose
        
    
    def train(self, X):
        super().train(X)
        self.num_users, self.num_items = self.full_matrix.shape
        self.P = np.random.normal(1/self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(1/self.K, size=(self.num_items, self.K))
        self.b_c = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = self.avg_rating
        
        self.R = np.array(self.full_matrix_dummy)
        self.user_to_idx = {user:idx for idx, user in enumerate(self.full_matrix.index)} 
        self.item_to_idx = {item:idx for idx, item in enumerate(self.full_matrix.columns)}
        rows, cols = self.R.nonzero()
        
        self.samples = [(row, col, self.R[row, col]) for row, col, in zip(rows, cols)]
        
        for i in range(self.iter):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse_inner(self.samples)

            
            if self.verbose == True:
                if (i+1) % 10 == 0:
                    print(f"Iteration {i+1} : RMSE : {rmse}")
            
        self.R = ((self.P.dot(self.Q.T)).T + self.b_c).T + self.b_i + self.b
        self.full_matrix = pd.DataFrame(self.R, index=self.full_matrix.index, columns=self.full_matrix.columns)
        
        return self
    
    def sgd(self):
        for i, j, r in self.samples:
            pred = self.get_prediction(i, j)
            e = (r - pred)
            
            self.b_c[i] += self.alpha * (e - self.beta * self.b_c[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])
            
    def rmse_inner(self, samples):
        errors = []
        for x, y, r in samples:
            pred = self.get_prediction(x, y)
            error = r - pred
            errors.append(error)
            
        rmse = np.sqrt(np.mean(np.array(errors) ** 2))
        
        return rmse
    
    def get_prediction(self, i, j):
        pred = self.b + self.b_c[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        
        return pred
    
    def rate(self, user, item):
        i = self.user_to_idx[user]
        j = self.item_to_idx[item]
        
        return self.R[i,j]

In [36]:
mfr = MF_Rrecommender(K=30, alpha=0.001, beta=0.02, iter=100, verbose=True)

In [37]:
mfr.train(ratings)

MF_Rrecommender train start.
Iteration 10 : RMSE : 1.4570713544227356
Iteration 20 : RMSE : 1.071982243857862
Iteration 30 : RMSE : 0.9402443810078414
Iteration 40 : RMSE : 0.8778102675420334
Iteration 50 : RMSE : 0.8422752362982135
Iteration 60 : RMSE : 0.8193103382076753
Iteration 70 : RMSE : 0.802985671803895
Iteration 80 : RMSE : 0.7904757071659065
Iteration 90 : RMSE : 0.7802970690952219
Iteration 100 : RMSE : 0.7716208876962363


In [38]:
mfr.rate('1', '947')

2.396145106383072

In [39]:
mfr.recommend('1', 10)

MF_Rrecommender recommend start.


['1', '479', '50', '5', '499', '498', '497', '496', '495', '494']

### Deep Learning (Neural Network)

In [40]:
class OneLayerNeuralNetwork_Recommender(Baseline_Recommender):
    
    def __init__(self, K=200, epochs=100, batch_size=256):
        self.K = K
        self.epochs = epochs
        self.batch_size = batch_size
        
    def train(self, X, refresh=True):
        super().train(X)
        self.user_to_idx = defaultdict(int)
        self.item_to_idx = defaultdict(int)
        
        def mapper(x, dict_val):
            if dict_val[x]:
                dict_val[x] = len(dict_val)
                
        self.ratings.user.apply(mapper, args=(self.user_to_idx,))
        self.ratings.item.apply(mapper, args=(self.item_to_idx,))
        
        self.ratings.user = self.ratings.user.apply(lambda x: self.user_to_idx[x])
        self.ratings.item = self.ratings.item.apply(lambda x: self.item_to_idx[x])
        M = self.ratings.user.unique().size + 1
        N = self.ratings.item.unique().size + 1
        self.mu = self.ratings.rate.mean()
        
        if refresh:
            user = Input(shape=(1,))
            item = Input(shape=(1,))
            
            P_embedding = Embedding(M, self.K, embeddings_regularizer=l2())(user)
            Q_embedding = Embedding(N, self.K, embeddings_regularizer=l2())(item)
            b_c_embedding = Embedding(M, 1, embeddings_regularizer=l2())(user)
            b_i_embedding = Embedding(N, 1, embeddings_regularizer=l2())(item)
            
            R = Dot(axes=2)([P_embedding, Q_embedding])
            R = Add()([R, b_c_embedding, b_i_embedding])
            R = Flatten()(R)
            
            self.model = Model(inputs=[user, item], outputs=R)
            self.model.compile(loss=self.rmse,
                               optimizer=SGD(),
                               metrics=[self.rmse, RootMeanSquaredError()])
            self.model.summary()
            self.model.fit(x=[self.ratings.user.values, self.ratings.item.values],
                           y=self.ratings.rate - self.mu,
                           epochs=self.epochs,
                           batch_size=self.batch_size,
                           validation_split=0.25)
            
            self.R = R
            
        return self
                
    def rate(self, user, item):
        u_idx = self.user_to_idx[user]
        i_idx = self.item_to_idx[item]
        
        return self.model.predict([np.array([u_idx]), np.array([i_idx])])[0][0] + self.mu
    
    @staticmethod
    def rmse(y_true, y_pred):
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
    
    def save(self, name):
        self.model.save(name)
        

In [41]:
olnnr = OneLayerNeuralNetwork_Recommender()

In [42]:
olnnr.train(ratings)

OneLayerNeuralNetwork_Recommender train start.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 200)       400         input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 200)       400         input_2[0][0]                    
_______________________________________________

Epoch 28/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1268 - rmse: 1.1265 - root_mean_squared_error: 1.1273 - val_loss: 1.1183 - val_rmse: 1.1181 - val_root_mean_squared_error: 1.1189
Epoch 29/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1258 - rmse: 1.1255 - root_mean_squared_error: 1.1261 - val_loss: 1.1182 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 30/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1255 - rmse: 1.1253 - root_mean_squared_error: 1.1260 - val_loss: 1.1182 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 31/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1240 - rmse: 1.1238 - root_mean_squared_error: 1.1245 - val_loss: 1.1181 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 32/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1227 - rmse: 1.1225 - root_mean_squared_error: 1.1233 - val_loss: 1.1182 - val_rmse: 1.118

293/293 [==============================] - 0s 1ms/step - loss: 1.1245 - rmse: 1.1245 - root_mean_squared_error: 1.1253 - val_loss: 1.1180 - val_rmse: 1.1181 - val_root_mean_squared_error: 1.1189
Epoch 68/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1286 - rmse: 1.1286 - root_mean_squared_error: 1.1294 - val_loss: 1.1180 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1189
Epoch 69/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1259 - rmse: 1.1259 - root_mean_squared_error: 1.1270 - val_loss: 1.1180 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 70/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1241 - rmse: 1.1241 - root_mean_squared_error: 1.1250 - val_loss: 1.1179 - val_rmse: 1.1179 - val_root_mean_squared_error: 1.1188
Epoch 71/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1284 - rmse: 1.1284 - root_mean_squared_error: 1.1294 - val_loss: 1.1180 - val_rmse: 1.1180 - val_root_

In [43]:
olnnr.rate('1', '947')

3.528151376628913

In [44]:
olnnr.recommend('1', 10)

OneLayerNeuralNetwork_Recommender recommend start.


['999', '372', '350', '818', '349', '348', '347', '346', '345', '344']

In [45]:
class DeepNeuralNetwork_Recommender(Baseline_Recommender):
    
    def __init__(self, K=200, epochs=100, batch_size=256):
        self.K = K
        self.epochs = epochs
        self.batch_size = batch_size

    def train(self, X):
        super().train(X)
        self.user_to_idx = defaultdict(int)
        self.item_to_idx = defaultdict(int)
        
        def mapper(x, dict_val):
            if not dict_val[x]:
                dict_val[x] = len(dict_val)
                
        self.ratings.user.apply(mapper, args=(self.user_to_idx,))
        self.ratings.item.apply(mapper, args=(self.item_to_idx,))
        
        self.ratings.user = self.ratings.user.apply(lambda x: self.user_to_idx[x])
        self.ratings.item = self.ratings.item.apply(lambda x: self.item_to_idx[x])
        M = self.ratings.user.unique().size + 1
        N = self.ratings.item.unique().size + 1
        self.mu = self.ratings.rate.mean()
        
        user = Input(shape=(1,), name='UserInput')
        item = Input(shape=(1,), name='ItemInput')
        
        P_embedding = Embedding(M, self.K, embeddings_regularizer=l2(), name='P')(user)
        Q_embedding = Embedding(N, self.K, embeddings_regularizer=l2(), name='Q')(item)
        user_bias = Embedding(M, 1, embeddings_regularizer=l2(), name='UserBias')(user)
        item_bias = Embedding(N, 1, embeddings_regularizer=l2(), name='ItemBias')(item)
        
        P_flat = Flatten(name='P_flat')(P_embedding)
        Q_flat = Flatten(name='Q_falt')(Q_embedding)
        user_bias_flat = Flatten(name='UserBias_flat')(user_bias)
        item_bias_flat = Flatten(name='ItemBias_flat')(item_bias)
        
        R = Concatenate(name='R_concat')([P_flat, Q_flat, user_bias_flat, item_bias_flat])
        
        for emb in [2048, 512, 128, 1]:
            R = Dense(emb)(R)
            R = Activation('linear')(R) if emb != 1 else R
            
        self.model = Model(inputs=[user, item], outputs=R)
        self.model.compile(loss=self.rmse,
                           optimizer=SGD(),
                           metrics=[self.rmse, RootMeanSquaredError()])
        self.model.summary()
        
        self.model.fit(
            x=[self.ratings.user.values, self.ratings.item.values],
            y=self.ratings.rate - self.mu,
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_split=0.25
        )
        
        self.R = R
        
        return self
    
    @staticmethod
    def rmse(y_true, y_pred):
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
    
    def save(self, name):
        self.model.save(name)


In [46]:
dnnr = DeepNeuralNetwork_Recommender()

In [47]:
dnnr.train(ratings)

DeepNeuralNetwork_Recommender train start.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
UserInput (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
ItemInput (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
P (Embedding)                   (None, 1, 200)       188800      UserInput[0][0]                  
__________________________________________________________________________________________________
Q (Embedding)                   (None, 1, 200)       336600      ItemInput[0][0]                  
_________________________________________________

293/293 [==============================] - 6s 19ms/step - loss: 1.4662 - rmse: 0.9339 - root_mean_squared_error: 0.9347 - val_loss: 1.4355 - val_rmse: 0.9472 - val_root_mean_squared_error: 0.9480
Epoch 20/100
293/293 [==============================] - 6s 19ms/step - loss: 1.4065 - rmse: 0.9316 - root_mean_squared_error: 0.9326 - val_loss: 1.3916 - val_rmse: 0.9559 - val_root_mean_squared_error: 0.9570
Epoch 21/100
293/293 [==============================] - 6s 19ms/step - loss: 1.3575 - rmse: 0.9337 - root_mean_squared_error: 0.9349 - val_loss: 1.3277 - val_rmse: 0.9388 - val_root_mean_squared_error: 0.9398
Epoch 22/100
293/293 [==============================] - 6s 19ms/step - loss: 1.3073 - rmse: 0.9289 - root_mean_squared_error: 0.9300 - val_loss: 1.2845 - val_rmse: 0.9373 - val_root_mean_squared_error: 0.9381
Epoch 23/100
293/293 [==============================] - 6s 19ms/step - loss: 1.2729 - rmse: 0.9350 - root_mean_squared_error: 0.9361 - val_loss: 1.2531 - val_rmse: 0.9428 - val_

293/293 [==============================] - 6s 19ms/step - loss: 0.9431 - rmse: 0.9255 - root_mean_squared_error: 0.9267 - val_loss: 0.9550 - val_rmse: 0.9381 - val_root_mean_squared_error: 0.9390
Epoch 59/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9457 - rmse: 0.9287 - root_mean_squared_error: 0.9299 - val_loss: 0.9556 - val_rmse: 0.9393 - val_root_mean_squared_error: 0.9401
Epoch 60/100
293/293 [==============================] - 6s 19ms/step - loss: 0.9440 - rmse: 0.9276 - root_mean_squared_error: 0.9286 - val_loss: 0.9560 - val_rmse: 0.9401 - val_root_mean_squared_error: 0.9412
Epoch 61/100
293/293 [==============================] - 6s 19ms/step - loss: 0.9455 - rmse: 0.9295 - root_mean_squared_error: 0.9306 - val_loss: 0.9518 - val_rmse: 0.9364 - val_root_mean_squared_error: 0.9373
Epoch 62/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9439 - rmse: 0.9284 - root_mean_squared_error: 0.9295 - val_loss: 0.9564 - val_rmse: 0.9415 - val_

293/293 [==============================] - 6s 20ms/step - loss: 0.9356 - rmse: 0.9239 - root_mean_squared_error: 0.9248 - val_loss: 0.9487 - val_rmse: 0.9373 - val_root_mean_squared_error: 0.9382
Epoch 98/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9380 - rmse: 0.9264 - root_mean_squared_error: 0.9271 - val_loss: 0.9551 - val_rmse: 0.9437 - val_root_mean_squared_error: 0.9445
Epoch 99/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9344 - rmse: 0.9228 - root_mean_squared_error: 0.9239 - val_loss: 0.9493 - val_rmse: 0.9381 - val_root_mean_squared_error: 0.9390
Epoch 100/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9407 - rmse: 0.9291 - root_mean_squared_error: 0.9302 - val_loss: 0.9488 - val_rmse: 0.9374 - val_root_mean_squared_error: 0.9384


In [48]:
dnnr.rate('1', '947')

3.52986

In [49]:
dnnr.recommend('1', 10)

DeepNeuralNetwork_Recommender recommend start.


['999', '372', '350', '818', '349', '348', '347', '346', '345', '344']

In [50]:
models = {
    "baseline": Baseline_Recommender(),
    "ubcf": Ubcf_Recommender(),
    "ubcf_knn": Ubcf_Knn_Recommender(K=30),
    "ubcf_knn_bias": Ubcf_Knn_Bias_Recommender(K=30),
    "ubcf_knn_bias_conf": Ubcf_Knn_Bias_Conf_Recommender(K=30, rating_cnt=2, min_rating=2),
    "ibdf": Ibcf_Recommender(),
    "mf": MF_Rrecommender(K=30, alpha=0.001, beta=0.02, iter=100, verbose=True),
    "olnnr": OneLayerNeuralNetwork_Recommender(),
    "dnnr": DeepNeuralNetwork_Recommender()
}

scores = dict()
for name, model in models.items():
    model.train(ratings)
    scores[name] = model.score()


Baseline_Recommender train start.
Baseline_Recommender evaluation start.
Baseline_Recommender 1.1211813036257785
Ubcf_Recommender train start.
Ubcf_Recommender evaluation start.
Ubcf_Recommender 0.9550036102729783
Ubcf_Knn_Recommender train start.
Ubcf_Knn_Recommender evaluation start.
Ubcf_Knn_Recommender 0.9006918048462648
Ubcf_Knn_Bias_Recommender train start.
Ubcf_Knn_Bias_Recommender evaluation start.
Ubcf_Knn_Bias_Recommender 0.8423591461215744
Ubcf_Knn_Bias_Conf_Recommender train start.
Ubcf_Knn_Bias_Conf_Recommender evaluation start.
Ubcf_Knn_Bias_Conf_Recommender 0.8513291936625513
Ibcf_Recommender train start.
Ibcf_Recommender evaluation start.
Ibcf_Recommender 0.976914219105312
MF_Rrecommender train start.
Iteration 10 : RMSE : 1.4588572006101135
Iteration 20 : RMSE : 1.0699662510040353
Iteration 30 : RMSE : 0.936733857040825
Iteration 40 : RMSE : 0.8737477076739347
Iteration 50 : RMSE : 0.8380543206841565
Iteration 60 : RMSE : 0.8151196764524185
Iteration 70 : RMSE : 0.7989

293/293 [==============================] - 0s 1ms/step - loss: 1.1288 - rmse: 1.1283 - root_mean_squared_error: 1.1291 - val_loss: 1.1185 - val_rmse: 1.1181 - val_root_mean_squared_error: 1.1189
Epoch 23/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1270 - rmse: 1.1265 - root_mean_squared_error: 1.1274 - val_loss: 1.1184 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 24/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1288 - rmse: 1.1284 - root_mean_squared_error: 1.1294 - val_loss: 1.1184 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 25/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1298 - rmse: 1.1294 - root_mean_squared_error: 1.1303 - val_loss: 1.1183 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 26/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1263 - rmse: 1.1260 - root_mean_squared_error: 1.1267 - val_loss: 1.1183 - val_rmse: 1.1180 - val_root_

293/293 [==============================] - 0s 1ms/step - loss: 1.1252 - rmse: 1.1252 - root_mean_squared_error: 1.1262 - val_loss: 1.1179 - val_rmse: 1.1179 - val_root_mean_squared_error: 1.1188
Epoch 62/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1273 - rmse: 1.1273 - root_mean_squared_error: 1.1280 - val_loss: 1.1180 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1189
Epoch 63/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1300 - rmse: 1.1300 - root_mean_squared_error: 1.1309 - val_loss: 1.1179 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1188
Epoch 64/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1274 - rmse: 1.1274 - root_mean_squared_error: 1.1283 - val_loss: 1.1180 - val_rmse: 1.1180 - val_root_mean_squared_error: 1.1189
Epoch 65/100
293/293 [==============================] - 0s 1ms/step - loss: 1.1257 - rmse: 1.1257 - root_mean_squared_error: 1.1264 - val_loss: 1.1179 - val_rmse: 1.1180 - val_root_

293/293 [==============================] - 0s 1ms/step - loss: 1.1286 - rmse: 1.1286 - root_mean_squared_error: 1.1295 - val_loss: 1.1181 - val_rmse: 1.1181 - val_root_mean_squared_error: 1.1189
OneLayerNeuralNetwork_Recommender evaluation start.
OneLayerNeuralNetwork_Recommender tf.Tensor(1.1231194793988977, shape=(), dtype=float64)
DeepNeuralNetwork_Recommender train start.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
UserInput (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
ItemInput (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
P (Embeddi

Epoch 18/100
293/293 [==============================] - 6s 20ms/step - loss: 1.5228 - rmse: 0.9260 - root_mean_squared_error: 0.9270 - val_loss: 1.4944 - val_rmse: 0.9473 - val_root_mean_squared_error: 0.9481
Epoch 19/100
293/293 [==============================] - 6s 20ms/step - loss: 1.4589 - rmse: 0.9267 - root_mean_squared_error: 0.9277 - val_loss: 1.4273 - val_rmse: 0.9392 - val_root_mean_squared_error: 0.9402
Epoch 20/100
293/293 [==============================] - 6s 20ms/step - loss: 1.4024 - rmse: 0.9276 - root_mean_squared_error: 0.9287 - val_loss: 1.3808 - val_rmse: 0.9452 - val_root_mean_squared_error: 0.9461
Epoch 21/100
293/293 [==============================] - 6s 20ms/step - loss: 1.3488 - rmse: 0.9251 - root_mean_squared_error: 0.9262 - val_loss: 1.3276 - val_rmse: 0.9388 - val_root_mean_squared_error: 0.9397
Epoch 22/100
293/293 [==============================] - 6s 20ms/step - loss: 1.3060 - rmse: 0.9277 - root_mean_squared_error: 0.9287 - val_loss: 1.2910 - val_rmse: 

293/293 [==============================] - 6s 20ms/step - loss: 0.9443 - rmse: 0.9260 - root_mean_squared_error: 0.9271 - val_loss: 0.9672 - val_rmse: 0.9496 - val_root_mean_squared_error: 0.9506
Epoch 58/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9436 - rmse: 0.9259 - root_mean_squared_error: 0.9271 - val_loss: 0.9604 - val_rmse: 0.9436 - val_root_mean_squared_error: 0.9444
Epoch 59/100
293/293 [==============================] - 6s 19ms/step - loss: 0.9455 - rmse: 0.9285 - root_mean_squared_error: 0.9296 - val_loss: 0.9572 - val_rmse: 0.9409 - val_root_mean_squared_error: 0.9419
Epoch 60/100
293/293 [==============================] - 6s 19ms/step - loss: 0.9392 - rmse: 0.9227 - root_mean_squared_error: 0.9236 - val_loss: 0.9592 - val_rmse: 0.9434 - val_root_mean_squared_error: 0.9442
Epoch 61/100
293/293 [==============================] - 6s 19ms/step - loss: 0.9464 - rmse: 0.9305 - root_mean_squared_error: 0.9314 - val_loss: 0.9516 - val_rmse: 0.9361 - val_

293/293 [==============================] - 6s 20ms/step - loss: 0.9420 - rmse: 0.9304 - root_mean_squared_error: 0.9314 - val_loss: 0.9487 - val_rmse: 0.9373 - val_root_mean_squared_error: 0.9382
Epoch 97/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9359 - rmse: 0.9242 - root_mean_squared_error: 0.9252 - val_loss: 0.9496 - val_rmse: 0.9382 - val_root_mean_squared_error: 0.9391
Epoch 98/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9400 - rmse: 0.9283 - root_mean_squared_error: 0.9294 - val_loss: 0.9499 - val_rmse: 0.9385 - val_root_mean_squared_error: 0.9394
Epoch 99/100
293/293 [==============================] - 6s 19ms/step - loss: 0.9376 - rmse: 0.9259 - root_mean_squared_error: 0.9269 - val_loss: 0.9490 - val_rmse: 0.9377 - val_root_mean_squared_error: 0.9386
Epoch 100/100
293/293 [==============================] - 6s 20ms/step - loss: 0.9366 - rmse: 0.9250 - root_mean_squared_error: 0.9263 - val_loss: 0.9487 - val_rmse: 0.9373 - val

In [51]:
scores

{'baseline': 1.1211813036257785,
 'ubcf': 0.9550036102729783,
 'ubcf_knn': 0.9006918048462648,
 'ubcf_knn_bias': 0.8423591461215744,
 'ubcf_knn_bias_conf': 0.8513291936625513,
 'ibdf': 0.976914219105312,
 'mf': 0.769151304135486,
 'olnnr': <tf.Tensor: shape=(), dtype=float64, numpy=1.1231194793988977>,
 'dnnr': <tf.Tensor: shape=(), dtype=float64, numpy=1.119885586477476>}